In [1]:
import json
from shapely.geometry import shape, GeometryCollection, Point
from typing import Tuple
#Documentation: https://shapely.readthedocs.io/en/stable/

In [2]:
#read geojson file for wojewodztwo level
with open('../../data/data_auxiliary/wojewodztwa-max.geojson', "r", encoding="utf-8") as f:
     wojewodztwa = json.load(f)
#read geojson file for powiat level
with open('../../data/data_auxiliary/powiaty-max.geojson', "r", encoding="utf-8") as f:
     powiaty = json.load(f)

In [3]:
def look_up(longitude: float, latitude:float ) -> Tuple[str,str] :
    
    point: Point = Point(longitude, latitude)
    
    wojewodztwo: str = 'not in PL'
    powiat: str = 'not in PL'
    
    for feature in wojewodztwa['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            wojewodztwo = feature['properties']['nazwa']
    
    for feature in powiaty['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            powiat = feature['properties']['nazwa']
        
    return wojewodztwo, powiat


In [4]:
info: Tuple[str,str] = look_up(20.975166, 50.173166 )

In [5]:
print(info)

('małopolskie', 'powiat dąbrowski')


# Checking powiat for weather stations

In [6]:
import pandas as pd
stations = pd.read_csv("../../data/data_processed/processed_weather_data_in_CSV_files/stations_with_powiat_voivod.csv")

In [7]:
for ind, i in enumerate(zip(stations['LON'],stations['LAT'],stations['state'],stations['county'])):
    pow_voi = look_up(i[0],i[1])
    pow_voi_old = ( i[2].split(' ')[1] if len(i[2].split(' ')) == 2 else i[2], i[3])
    if pow_voi[1] != pow_voi_old[1]:
        stations.loc[ind,('county')] = pow_voi[1]
        print(f"LAT: {i[1]}, LON: {i[0]}, OLD VOIVOD/POWIAT: {pow_voi_old[0],pow_voi_old[1]}, NEW VOIVOD/POWIAT: {pow_voi[0],pow_voi[1]}")

LAT: 50.78333333, LON: 15.78333333, OLD VOIVOD/POWIAT: ('dolnośląskie', 'powiat karkonoski'), NEW VOIVOD/POWIAT: ('dolnośląskie', 'powiat jeleniogórski')
LAT: 51.96666667, LON: 20.16666667, OLD VOIVOD/POWIAT: ('łódzkie', 'Powiat skierniewicki'), NEW VOIVOD/POWIAT: ('łódzkie', 'powiat Skierniewice')
LAT: 52.28333333, LON: 20.96666667, OLD VOIVOD/POWIAT: ('mazowieckie', 'Warszawa'), NEW VOIVOD/POWIAT: ('mazowieckie', 'powiat Warszawa')
LAT: 52.21944444, LON: 20.99916667, OLD VOIVOD/POWIAT: ('mazowieckie', 'Warszawa'), NEW VOIVOD/POWIAT: ('mazowieckie', 'powiat Warszawa')
LAT: 54.38333333, LON: 18.46666667, OLD VOIVOD/POWIAT: ('pomorskie', 'Gdańsk County'), NEW VOIVOD/POWIAT: ('pomorskie', 'powiat Gdańsk')
LAT: 54.33333333, LON: 18.93333333, OLD VOIVOD/POWIAT: ('pomorskie', 'Gdańsk County'), NEW VOIVOD/POWIAT: ('pomorskie', 'powiat Gdańsk')
LAT: 50.90027778, LON: 15.78888889, OLD VOIVOD/POWIAT: ('dolnośląskie', 'Jelenia Góra County'), NEW VOIVOD/POWIAT: ('dolnośląskie', 'powiat Jelenia Gó

In [8]:
import random

for ind, i in enumerate(zip(stations['LON'],stations['LAT'],stations['state'],stations['county'])):
    pow_voi = (i[2], i[3])
    while pow_voi[1] == 'not in PL':
        new_coords = (i[0] + 0.1*(2*random.random()-1), i[1] + 0.1*(2*random.random()-1))
        pow_voi = look_up(*new_coords)
        if pow_voi != ('not in PL', 'not in PL'):
            print(i, "->", pow_voi)
            stations.loc[ind, ('state','county')] = ['województwo ' + pow_voi[0], pow_voi[1]]

(14.53777778, 52.36527778, 'Brandenburg', 'not in PL') -> ('lubuskie', 'powiat słubicki')
(15.73, 50.73, 'Severovýchod', 'not in PL') -> ('dolnośląskie', 'powiat jeleniogórski')


In [11]:
stations.to_csv("../../data/data_processed/processed_weather_data_in_CSV_files/stations_with_powiat_voivod_NEW.csv")